# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 21 2022 11:55AM,242441,10,8299743,Eywa Pharma Inc.,Released
1,Jul 21 2022 11:55AM,242441,10,8299744,Eywa Pharma Inc.,Released
2,Jul 21 2022 11:55AM,242441,10,8299746,Eywa Pharma Inc.,Released
3,Jul 21 2022 11:55AM,242441,10,8299747,Eywa Pharma Inc.,Released
4,Jul 21 2022 11:55AM,242441,10,8299749,Eywa Pharma Inc.,Released
5,Jul 21 2022 11:55AM,242441,10,8299750,Eywa Pharma Inc.,Released
6,Jul 21 2022 11:55AM,242441,10,8299751,Eywa Pharma Inc.,Released
7,Jul 21 2022 11:55AM,242441,10,8299753,Eywa Pharma Inc.,Released
8,Jul 21 2022 11:55AM,242441,10,8299755,Eywa Pharma Inc.,Released
9,Jul 21 2022 11:55AM,242441,10,8300297,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,242434,Released,1
42,242437,Executing,1
43,242439,Released,13
44,242440,Released,1
45,242441,Released,13


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242434,NaN,NaN,1.0
242437,NaN,1.0,NaN
242439,NaN,NaN,13.0
242440,NaN,NaN,1.0
242441,NaN,NaN,13.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242325,0.0,0.0,1.0
242328,0.0,0.0,1.0
242336,0.0,0.0,1.0
242340,0.0,0.0,1.0
242348,0.0,18.0,60.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242325,0,0,1
242328,0,0,1
242336,0,0,1
242340,0,0,1
242348,0,18,60


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242325,0,0,1
1,242328,0,0,1
2,242336,0,0,1
3,242340,0,0,1
4,242348,0,18,60


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242325,,,1
1,242328,,,1
2,242336,,,1
3,242340,,,1
4,242348,,18,60


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 21 2022 11:55AM,242441,10,Eywa Pharma Inc.
13,Jul 21 2022 11:49AM,242440,19,"AdvaGen Pharma, Ltd"
14,Jul 21 2022 11:43AM,242439,20,Alumier Labs Inc.
27,Jul 21 2022 11:36AM,242437,12,Hush Hush
28,Jul 21 2022 11:30AM,242434,10,Bio-PRF
29,Jul 21 2022 11:22AM,242433,10,Bio-PRF
39,Jul 21 2022 11:14AM,242422,15,"Person & Covey, Inc."
54,Jul 21 2022 11:04AM,242431,21,"NBTY Global, Inc."
55,Jul 21 2022 10:59AM,242429,15,"Alliance Pharma, Inc."
67,Jul 21 2022 10:57AM,242425,15,"Virtus Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 21 2022 11:55AM,242441,10,Eywa Pharma Inc.,,,13
1,Jul 21 2022 11:49AM,242440,19,"AdvaGen Pharma, Ltd",,,1
2,Jul 21 2022 11:43AM,242439,20,Alumier Labs Inc.,,,13
3,Jul 21 2022 11:36AM,242437,12,Hush Hush,,1,
4,Jul 21 2022 11:30AM,242434,10,Bio-PRF,,,1
5,Jul 21 2022 11:22AM,242433,10,Bio-PRF,,,10
6,Jul 21 2022 11:14AM,242422,15,"Person & Covey, Inc.",,14,1
7,Jul 21 2022 11:04AM,242431,21,"NBTY Global, Inc.",,,1
8,Jul 21 2022 10:59AM,242429,15,"Alliance Pharma, Inc.",,4,8
9,Jul 21 2022 10:57AM,242425,15,"Virtus Pharmaceuticals, LLC",,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 11:55AM,242441,10,Eywa Pharma Inc.,13,,
1,Jul 21 2022 11:49AM,242440,19,"AdvaGen Pharma, Ltd",1,,
2,Jul 21 2022 11:43AM,242439,20,Alumier Labs Inc.,13,,
3,Jul 21 2022 11:36AM,242437,12,Hush Hush,,1,
4,Jul 21 2022 11:30AM,242434,10,Bio-PRF,1,,
5,Jul 21 2022 11:22AM,242433,10,Bio-PRF,10,,
6,Jul 21 2022 11:14AM,242422,15,"Person & Covey, Inc.",1,14,
7,Jul 21 2022 11:04AM,242431,21,"NBTY Global, Inc.",1,,
8,Jul 21 2022 10:59AM,242429,15,"Alliance Pharma, Inc.",8,4,
9,Jul 21 2022 10:57AM,242425,15,"Virtus Pharmaceuticals, LLC",3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 11:55AM,242441,10,Eywa Pharma Inc.,13,,
1,Jul 21 2022 11:49AM,242440,19,"AdvaGen Pharma, Ltd",1,,
2,Jul 21 2022 11:43AM,242439,20,Alumier Labs Inc.,13,,
3,Jul 21 2022 11:36AM,242437,12,Hush Hush,,1,
4,Jul 21 2022 11:30AM,242434,10,Bio-PRF,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 11:55AM,242441,10,Eywa Pharma Inc.,13.0,NaN,NaN
1,Jul 21 2022 11:49AM,242440,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
2,Jul 21 2022 11:43AM,242439,20,Alumier Labs Inc.,13.0,NaN,NaN
3,Jul 21 2022 11:36AM,242437,12,Hush Hush,NaN,1.0,NaN
4,Jul 21 2022 11:30AM,242434,10,Bio-PRF,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 11:55AM,242441,10,Eywa Pharma Inc.,13.0,0.0,0.0
1,Jul 21 2022 11:49AM,242440,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
2,Jul 21 2022 11:43AM,242439,20,Alumier Labs Inc.,13.0,0.0,0.0
3,Jul 21 2022 11:36AM,242437,12,Hush Hush,0.0,1.0,0.0
4,Jul 21 2022 11:30AM,242434,10,Bio-PRF,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3151222,148.0,24.0,8.0
12,484773,1.0,1.0,0.0
15,969700,14.0,18.0,0.0
16,1212032,8.0,0.0,0.0
19,1211993,16.0,2.0,0.0
20,484787,73.0,18.0,0.0
21,1939029,8.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3151222,148.0,24.0,8.0
1,12,484773,1.0,1.0,0.0
2,15,969700,14.0,18.0,0.0
3,16,1212032,8.0,0.0,0.0
4,19,1211993,16.0,2.0,0.0
5,20,484787,73.0,18.0,0.0
6,21,1939029,8.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,148.0,24.0,8.0
1,12,1.0,1.0,0.0
2,15,14.0,18.0,0.0
3,16,8.0,0.0,0.0
4,19,16.0,2.0,0.0
5,20,73.0,18.0,0.0
6,21,8.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,148.0
1,12,Released,1.0
2,15,Released,14.0
3,16,Released,8.0
4,19,Released,16.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,8.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,24.0,1.0,18.0,0.0,2.0,18.0,0.0
Released,148.0,1.0,14.0,8.0,16.0,73.0,8.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,8.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,24.0,1.0,18.0,0.0,2.0,18.0,0.0
2,Released,148.0,1.0,14.0,8.0,16.0,73.0,8.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,8.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,24.0,1.0,18.0,0.0,2.0,18.0,0.0
2,Released,148.0,1.0,14.0,8.0,16.0,73.0,8.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()